# Properties
## Access data only through special logic

In python, or in any programming, sometimes we want to make a restriction that accesses to a piece of data can only be done by following certain prescribed logic, so the integrity of the data is well enforced.  

This can usually be done by using getter and setter, and implementing the logic of access in the getter and setter.

In [ ]:
# Version 1, no protection of the data
class person1:
  def __init__(self, name, age=0):
    self.name = name
    self.age = age

a1 = person1('John', 30)

# If we want to update John's age, we can only update expicitly
a1.age = 31

# But there can be errors in the program that lead to unreasonable values, for example
b1 = person1('Peter')
b1.age = -1 #This is unreasonable, but the program will accept it


In [ ]:
# Version 2, using getter and setter
# This usually done by assigning a "private varialbe" name to the data
# In python, a data attribute that starts with an underscore is cnosidered 
# a private variable by convention
class person2:
  def __init__(self, name, age=0):
    self.name = name
    self._age = age

  def get_age(self):
    print("{}'s age is {}".format(self.name, self._age))
    return self._age

  def set_age(self, age):
    if age < 0:
      print('Age cannot be negative')
    elif age > 200:
      print('Unreasonable value: {}}is too big as a value for age.')
    else:
      self._age = age
      print("{}'s age is successfully updated".format(self.name))

a2 = person2('John')
a2.set_age(30)

b2 = person2('Peter')
b2.set_age(-10)  # This update will be rejected

a2.get_age()
b2.get_age()

In [ ]:
# You can still change object data members to unreasonable values by direct access
b2._age = -3
b2.get_age()

Peter's age is -3


-3

## Problems with only using getters and setters

The above code using getter and setter works fine to protect the integrity of the data 'age' now.  However, there are still two drawbacks:
1. The code now becomes more verbose.
2. Much more seriously, if there are old code that were already written using the plain style of access, we now have to modify all that old code.  
  a. Note sometimes it is just a matter of being cumbersome, but sometimes we simpe do have no way of modifying the code.  For example, we may want to make our code to work with a large piece of code written by someone else.

## The property class
Python offers a convenient way to solve the problem in this situation: using preperty class. 

In [ ]:
# Use property class
class person3:
  def __init__(self, name, age=0):
    self.name = name
    self.age = age

  def get_age(self):
    print("{}'s age is {}".format(self.name, self._age))
    return self._age

  def set_age(self, age):
    if age < 0:
      raise ValueError('Age cannot be negative')
    elif age > 200:
      raise ValueError('Unreasonable value: {} is too big as a value for age.'.format(age))
    else:
      self._age = age
      print("{}'s age is successfully updated as {}".format(self.name, age))
  
  age = property(get_age, set_age)

# Note that we now no longer need to call get_age() and set_age() explicitly
a3 = person3('John', 23)
print(a3.age)

John's age is successfully updated as 23
John's age is 23
23


In [ ]:
a3.age = -5

In [ ]:
a3.age = 500

In [ ]:
print(a3.age)
a3.age = 30
print(a3.age)

In [ ]:
# Now we can test something even more powerful
b3 = person3('Peter', -20)
print(b3.age)

# What is property?
property is actually a class: `class property`.  That is, in the previous code, when we run
```
age = property(get_temp, set_temp)
```
we actually create a property object, and assign that object to `age`.


And
1. When we create a property object, the first argument of the object creation call is used as the getter, and the second argument the setter.
2. When we read the value of this object, its getter function is called.
3. When we assign a value to this object, its setter function is called.

The full creation interface of property is as follows:
```
 |  property(fget=None, fset=None, fdel=None, doc=None) -> property attribute
 |  
 |  fget is a function to be used for getting an attribute value, and likewise
 |  fset is a function for setting, and fdel a function for del'ing, an
 |  attribute.  Typical use is to define a managed attribute x:
 |  
 |  class C(object):
 |      def getx(self): return self._x
 |      def setx(self, value): self._x = value
 |      def delx(self): del self._x
 |      x = property(getx, setx, delx, "I'm the 'x' property.")
 |  
```


In [ ]:
help(property)

## Property decorator
Finally, we can implement the above property logic using decorator mechanism.  This can make the code more readable.

Python help describes the property decorator as follows:
```
 |  Decorators make defining new properties or modifying existing ones easy:
 |  
 |  class C(object):
 |      @property
 |      def x(self):
 |          "I am the 'x' property."
 |          return self._x
 |      @x.setter
 |      def x(self, value):
 |          self._x = value
 |      @x.deleter
 |      def x(self):
 |          del self._x
```

In [ ]:
# Use property class
class person4:
  def __init__(self, name, age=0):
    self.name = name
    self.age = age

  @property
  def age(self):
    print("{}'s age is {}".format(self.name, self._age))
    return self._age

  @age.setter
  def age(self, age):
    if age < 0:
      raise ValueError('Age cannot be negative')
    elif age > 200:
      raise ValueError('Unreasonable value: {} is too big as a value for age.'.format(age))
    else:
      self._age = age
      print("{}'s age is successfully updated as {}".format(self.name, age))
  
  # we also add an age deleter to make thing more complete
  @age.deleter
  def age(self):
    del self._age
  

# Note that we now no longer need to call get_age() and set_age() explicitly
a4 = person4('John')
print(a4.age)

b4 = person4('Peter')
b4.age = -10

In [ ]:
c4 = person4('Mary', 800)

### Brief explanation of property decorator

The above code work well and somewhat magically.

The following is a brief explanation.

1. @property
```@property
  def age(self):
    print("{}'s age is {}".format(self.name, self._age))
    return self._age
```
What this code does is to enable the function attribute age() to be read-accessed like a data attribute.  For ths to work, you must make sure age() returns something.  That's all.
2. @age.setter
```
@age.setter
  def age(self, age):
    if age < 0:
      raise ValueError('Age cannot be negative')
    elif age > 200:
      raise ValueError('Unreasonable value: {} is too big as a value for age.'.format(age))
    else:
      self._age = age
      print("{}'s age is successfully updated as {}".format(self.name, age))
``` 
This code enables the name age to be write-accessed like a data member. Implement your data integrity checking logic here.  And after the checking, you can update the (any) data your want to update here.  (To be more accurate, this code first redefine the function age() as update operation, then associates this newly redefined age() function with the action of write-access to the name age.  


For a thorough understanding of how this work in depth, you can study the topic of python **descriptor**.